In [ ]:
# If there is a ModuleNotFound error, try uncommenting
# and running the below line
#%pip install openai

It's assumed that you have an OpenAI API key and that this key is assigned to the environment variable `OPENAI_API_KEY`. If these assumptions don't hold, change line 15 in `openai_client.py` accordingly.

In the code below, we instantiate the OpenAIClient with a model. If you're unsure of which models are available, you can instantiate the client without passing it a model then run `client.list_models()` to get back a list of available models:
```python
client = OpenAIClient() # No model given
available_models = client.list_models() # Returns a List[dict] of available models
some_model = ... # pick some model from the `available_models`
client.model = some_model
```

In [1]:
from openai_client import OpenAIClient

# If you're unsure of which models are available,
# you can instantiate the client without passing it a model
# then run `client.list_models()` to get back a list of 
# available models
model = "gpt-4-1106-preview"
gpt4 = OpenAIClient(model)

The example `request_completion` implementation does not take every available parameter for the chat endpoint. 

See here for endpoint details: https://platform.openai.com/docs/api-reference/chat

See the `openai_client.py` for the example code

In [2]:
# Set up some parameters we'll use for the chat endpoint
temp = 0.7
max_tokens = 3_096

# Cf. logos_app.py for details on the `search_logos_library` function
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_logos_library",
            "description": "Searches the LogosApplication library for a given query and returns a JSON string of the search results",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search term to query in the LogosApplication library",
                    },
                },
                "required": ["query"],
            },
        },
    }
]

sys_msg = "You are a helpful assistant."
gpt4.set_sys_msg(sys_msg)

The `OpenAIClient` has a `messages` list that is an empty list upon instantiation. This will be sent to the OpenAI API when using the `chat.completion.create` endpoint.

To check the messages list that will be sent use the `display_messages` method:
```python
gpt4.display_messages()
```
Clear the messages with the `clear_messages` method:
```python
gpt4.clear_messages()
```
The `messages` list will be automatically updated with the assistant's response after sending.

In the code above we set the standard example for a system message: 'You are a helpful assistant.' 

Add a user message:

In [3]:
# The AI won't choose to make use of Logos (call our function) if it's not deemed relevant, 
# so we give a relevant example prompt. 
user_msg = "What resources in my Logos library mention 'idolatry'?"
gpt4.add_user_msg(user_msg)

Finally, execute the request. 

Note that making use of a tool (calling a function as it currently stands) results in two requests: 
1) The initial request where the AI will decide to call a function.
2) Once we have the result(s) of the function, another request containing the function result(s) so that the AI can make use of the information.

Any second request is handled automatically within the `request_completion` method. 

When using the Logos COM API, we have to launch the Logos application. Logos startup in itself can take several seconds. Thus, the entire process of sending a request, getting a response to call a function, launching Logos, executing our specific Logos task, sending another request to the AI with the function results, and then getting our final response from the AI can take a while! You can speed this process up by having Logos already running.

In [4]:
gpt4.request_completion(temp=temp, max_tokens=max_tokens, tools=tools)

Here are some resources in your Logos library that mention 'idolatry':

1. Christian History Magazine—Issue 14: Money in Christian History: Part I
2. Calvin’s Calvinism: A Defence of the Secret Providence of God
3. 1 and 2 Kings
4. Dictionary of Scripture and Ethics
5. The Purpose of God: Ephesians
6. Themelios: Volume 24, No. 3, June 1999
7. Ante-Nicene Fathers 3: Latin Christianity: Its Founder, Tertullian
8. God of Our Fathers: Classical Theism for the Contemporary Church

These resources should provide a variety of perspectives and in-depth discussions on the topic of idolatry.
None
